In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import datetime
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score

# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

In [2]:
# Scale pixel values to between 0 and 1
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Reshape
x_train = np.reshape(x_train, (x_train.shape[0], 28, 28, 1))
x_test = np.reshape(x_test, (x_test.shape[0], 28, 28, 1))
# Split data into 90% training and 10% validation sets
split = int(0.9 * len(x_train))
x_train, x_val = x_train[:split], x_train[split:]
y_train, y_val = y_train[:split], y_train[split:]

In [4]:
# Define LeNet model architecture
model = keras.Sequential([
        layers.Conv2D(filters=6, kernel_size=(5, 5), activation="relu", input_shape=(28, 28, 1)),
        layers.AveragePooling2D(),
        layers.Conv2D(filters=16, kernel_size=(5, 5), activation="relu"),
        layers.AveragePooling2D(),
        layers.Flatten(),
        layers.Dense(units=120, activation="relu"),
        layers.Dense(units=84, activation="relu"),
        layers.Dense(units=10, activation="softmax"),   
])

# Compile the model
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [11]:
# Define tensorboard logs directory
log_dir = 'C:/Users/kstre_3ikvnbf/CS 4375/Project/MnistLeNet'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
# Train the model
model.fit(x_train, y_train, batch_size=64, epochs=30, validation_data=(x_val, y_val), callbacks=[tensorboard_callback])

In [15]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(x_test, y_test)

# Generate predictions on the test set
y_pred = model.predict(x_test)

# Convert predicted probabilities to predicted class labels
y_pred_classes = np.argmax(y_pred, axis=1)

313/313 [==============================] - 2s 6ms/step


In [16]:
# Compute evaluation metrics
conf_matrix = confusion_matrix(y_test, y_pred_classes)
precision = precision_score(y_test, y_pred_classes, average='macro')
recall = recall_score(y_test, y_pred_classes, average='macro')
f1score = f1_score(y_test, y_pred_classes, average='macro')
auc_roc = roc_auc_score(y_test, y_pred, multi_class='ovr')
    
# Print the evaluation metrics
print("Confusion Matrix:\n", conf_matrix)
print("Test Loss:", test_loss)
print("Accuracy:", test_acc)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1score)
print("AU-ROC:", auc_roc)

Confusion Matrix:
 [[ 975    0    0    0    1    0    1    1    1    1]
 [   0 1135    0    0    0    0    0    0    0    0]
 [   0    1 1026    0    1    0    0    4    0    0]
 [   0    0    0 1008    0    1    0    0    1    0]
 [   0    0    1    0  971    0    4    0    0    6]
 [   2    1    0   10    0  875    2    0    0    2]
 [   3    4    0    1    0    2  947    0    1    0]
 [   0    4    3    0    0    0    0 1020    0    1]
 [   2    0    3    4    0    1    1    0  961    2]
 [   0    1    0    1    8    2    0    2    2  993]]
Test Loss: 0.0468890555202961
Accuracy: 0.991100013256073
Precision: 0.9911558370377394
Recall: 0.9908375219491562
F1-score: 0.9909833768399323
AU-ROC: 0.9999355071090088


In [10]:
# Log evaluation metrics to TensorBoard
test_writer = tf.summary.create_file_writer(log_dir)
with test_writer.as_default():
    tf.summary.scalar('Accuracy', test_acc, step=0)
    tf.summary.scalar('Precision', precision, step=0)
    tf.summary.scalar('Recall', recall, step=0)
    tf.summary.scalar('F1-score', f1score, step=0)
    tf.summary.scalar('AU-ROC', auc_roc, step=0)

In [ ]:
# tensorboard --logdir_spec=FashionMLP:"C:/Users/kstre_3ikvnbf/CS 4375/Project/FashionMLP",FashionLeNet:"C:/Users/kstre_3ikvnbf/CS 4375/Project/FashionLeNet",MnistLeNet:"C:/Users/kstre_3ikvnbf/CS 4375/Project/MnistLeNet",MnistMLP:"C:/Users/kstre_3ikvnbf/CS 4375/Project/MnistMLP"